In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,LabelEncoder
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score,make_scorer
from sklearn.tree import DecisionTreeClassifier
import joblib
from sklearn.tree import DecisionTreeClassifier

In [28]:
data=pd.read_csv("NPI/data.csv")

In [30]:
data.head()

,score,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q34,Q35,Q36,Q37,Q38,Q39,Q40,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50


In [34]:
X=data.drop(columns=['score','elapse','gender','age'])

In [35]:
y=data['score']

In [81]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
pipe=Pipeline([
    ('scale',StandardScaler()),
    ('model',LogisticRegression(max_iter=1000))
])
pipe.fit(X_train,y_train)
pred=pipe.predict(X_test)
score=accuracy_score(y_test,pred)
print('Actual:',y_test)
print('Predicted:',pred)
print('accuracy:',score)
pipe.get_params()

Actual: 3716     25
10548     6
8795     24
4904      8
983       9
         ..
5986      6
11047    12
2550      3
6960      5
8292      4
Name: score, Length: 2249, dtype: int64
Predicted: [25  5 28 ...  4  5  4]
accuracy: 0.42507781236104936


{'memory': None,
 'steps': [('scale', StandardScaler()),
  ('model', LogisticRegression(max_iter=1000))],
 'verbose': False,
 'scale': StandardScaler(),
 'model': LogisticRegression(max_iter=1000),
 'scale__copy': True,
 'scale__with_mean': True,
 'scale__with_std': True,
 'model__C': 1.0,
 'model__class_weight': None,
 'model__dual': False,
 'model__fit_intercept': True,
 'model__intercept_scaling': 1,
 'model__l1_ratio': None,
 'model__max_iter': 1000,
 'model__multi_class': 'auto',
 'model__n_jobs': None,
 'model__penalty': 'l2',
 'model__random_state': None,
 'model__solver': 'lbfgs',
 'model__tol': 0.0001,
 'model__verbose': 0,
 'model__warm_start': False}

In [ ]:
grid=GridSearchCV(estimator=pipe,
                 param_grid={'model__class_weight':[{0:1,1:v} for v in np.linspace(1,20,30)]},
                 cv=10,
                 scoring={'precision':make_scorer(precision_score),'recall':make_scorer(recall_score)},
                 refit='precision')
grid.fit(X_train,y_train)
gpred=grid.predict(X_test)
# precision_score(y_test,pred)y
print('accuracy',accuracy_score(y_test,gpred))

In [55]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014328,0.001237,0.386331,0.059579,1,{'model__n_neighbors': 1},0.166444,0.164443,0.166778,0.165888,0.001031,1
1,0.009352,0.000459,0.360660,0.045475,2,{'model__n_neighbors': 2},0.112408,0.104403,0.112408,0.109740,0.003774,2
2,0.009003,0.000004,0.346982,0.001623,3,{'model__n_neighbors': 3},0.098065,0.090060,0.096398,0.094841,0.003449,3
3,0.009006,0.000006,0.425661,0.002871,4,{'model__n_neighbors': 4},0.094730,0.091061,0.094063,0.093284,0.001596,4
4,0.010317,0.000447,0.424338,0.003271,5,{'model__n_neighbors': 5},0.090727,0.084390,0.094063,0.089726,0.004012,6
5,0.009010,0.000016,0.422674,0.000952,6,{'model__n_neighbors': 6},0.095063,0.089393,0.093396,0.092617,0.002379,5
6,0.009987,0.000819,0.426339,0.008723,7,{'model__n_neighbors': 7},0.091394,0.083055,0.091394,0.088615,0.003931,7
7,0.009012,0.000018,0.429642,0.003110,8,{'model__n_neighbors': 8},0.079386,0.079386,0.093062,0.083945,0.006447,8
8,0.009001,0.000816,0.423679,0.008731,9,{'model__n_neighbors': 9},0.075717,0.074383,0.083722,0.077941,0.004124,9
9,0.008989,0.000015,0.435010,0.002905,10,{'model__n_neighbors': 10},0.073382,0.067712,0.077051,0.072715,0.003842,10
